# Tapas in Vancouver

## Introduction

The goal of this exercise is to find the best location in Vancouver for a new "tapas" restaurant (https://en.wikipedia.org/wiki/Tapas). 

Imagine a company which wants to open a new restaurant in Vancouver. This restaurant will serve only tapas in a "Spanish Bar" way. As is not your usual restaurant, it becomes fundamental to choose carefully the neighborhood. 

We use data from wikipedia to define Vancouver negihborhoods and data from Foursquares to gather informationa aboit venues already established; postal codes from Vancouver to delimit the neighboorhoods and Foursquare data to locate restaurants in those neighborhoods, and the restaurants main cuisine.

* Clean Vancouver postal codes.
Get only the postal codes from Vancouver neighborhoods from "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V". 

We need to find a neighborhood with a lot of different restaurants and/or a neighborhood with an important spanish community (maybe latino restaurants )

We need to find a neighborhood with a lot of differente restaurants and/or a neighborhood with an importan spanish community (maybe latino restaurants )

In [1]:
# Foursquare data. !!!!!!!
CLIENT_ID = # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
ACCESS_TOKEN = 
LIMIT = 300 

In [2]:
# import the libraries we're going to need
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

# to visualize maps
import folium
# create maps using latitude and longitude values
from geopy.geocoders import Nominatim as Nominatimgeopy

import matplotlib.cm as cm
import matplotlib.colors as colors

# clustering
from sklearn.cluster import KMeans


## Neighborhoods in Vancouver

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V' # postal codes in British Columbia not only Vancouver
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")
tabla = soup.find('table') # It's the first table on the page
neighborhoods_postalcode = pd.DataFrame(columns=["Postal Code","Borough","Neighborhood"])

In [4]:
for row in tabla.tbody.find_all("tr"):
    col = row.find_all("td") 
    if (col != []):
        for i in range(len(col)):
            if "Vancouver" in col[i].text:
                postalcode = col[i].text[:3]
                borough = 'Vancouver'
                if col[i].text[3].strip() == 'V':
                    neighborhood = col[i].text[3:].split('(')[1].strip()[:-1]
   #             borough = col[i].text[3:].split('\n')[0].strip() 
   #             print(borough)
   #             neighborhood = col[i].text[(3+len(borough)):].strip()
   #             print(neighborhood)
   #             neighborhood = col[i].text[3:].split('(')[1].rstrip()
                neighborhoods_postalcode = neighborhoods_postalcode.append({"Postal Code":postalcode, "Borough":borough,"Neighborhood":neighborhood}, ignore_index=True)

In [5]:
neighborhoods_postalcode.head(6)

,Postal Code,Borough,Neighborhood
0,V6A,Vancouver,Strathcona / Chinatown / Downtown Eastside
1,V6B,Vancouver,NE Downtown / Gastown / Harbour Centre / Inter...
2,V6C,Vancouver,Waterfront / Coal Harbour / Canada Place
3,V6E,Vancouver,SE West End / Davie Village
4,V6G,Vancouver,NW West End / Stanley Park
5,V7G,Vancouver,NW West End / Stanley Park


In [6]:
neighborhoods_postalcode.shape

(44, 3)

Encontrar las coordenadas en base a los códigos postales de Vancouver 

In [7]:
from pgeocode import Nominatim
nomi = Nominatim('ca')
postal_codes = neighborhoods_postalcode['Postal Code'].tolist()
Latitude =[nomi.query_postal_code(postalcode)['latitude'] for postalcode in postal_codes]
Longitude = [nomi.query_postal_code(postalcode)['longitude'] for postalcode in postal_codes]
coordinates = pd.DataFrame([postal_codes, Latitude, Longitude]).transpose()
coordinates.columns=['Postal Code','Latitude','Longitude']
coordinates.head()

,Postal Code,Latitude,Longitude
0,V6A,49.2779,-123.091
1,V6B,49.2788,-123.114
2,V6C,49.2866,-123.116
3,V6E,49.2833,-123.13
4,V6G,49.299,-123.141


In [8]:
neighborhoods = neighborhoods_postalcode.join(coordinates.set_index('Postal Code'), on='Postal Code')
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,V6A,Vancouver,Strathcona / Chinatown / Downtown Eastside,49.2779,-123.091
1,V6B,Vancouver,NE Downtown / Gastown / Harbour Centre / Inter...,49.2788,-123.114
2,V6C,Vancouver,Waterfront / Coal Harbour / Canada Place,49.2866,-123.116
3,V6E,Vancouver,SE West End / Davie Village,49.2833,-123.13
4,V6G,Vancouver,NW West End / Stanley Park,49.299,-123.141


In [9]:
address = 'Vancouver, BC'

geolocator = Nominatimgeopy(user_agent='bc_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))
map_vancouver = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vancouver)  
    
map_vancouver

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,  lat, lng, ACCESS_TOKEN, VERSION,  radius, LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
vancouver_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Strathcona / Chinatown / Downtown Eastside
NE Downtown / Gastown / Harbour Centre / International Village / Victory Square / Yaletown
Waterfront / Coal Harbour / Canada Place
SE West End / Davie Village
NW West End / Stanley Park
NW West End / Stanley Park
West Fairview / Granville Island / NE Shaughnessy
West Fairview / Granville Island / NE Shaughnessy
NW Shaughnessy / East Kitsilano / Quilchena
NW Shaughnessy / East Kitsilano / Quilchena
North Hastings-Sunrise
Central Kitsilano / Greektown
Central Kitsilano / Greektown
North Grandview-Woodland
NW Arbutus Ridge / NE Dunbar-Southlands
NW Arbutus Ridge / NE Dunbar-Southlands
South Hastings-Sunrise / North Renfrew-Collingwood
South Shaughnessy / NW Oakridge / NE Kerrisdale / SE Arbutus Ridge
South Shaughnessy / NW Oakridge / NE Kerrisdale / SE Arbutus Ridge
South Grandview-Woodland / NE Kensington-Cedar Cottage
West Kerrisdale / South Dunbar-Southlands / Musqueam
West Kerrisdale / South Dunbar-Southlands / Musqueam
SE Kensington-Cedar C

In [12]:
vancouver_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Strathcona / Chinatown / Downtown Eastside,49.2779,-123.0908,Union Market,49.277371,-123.086989,Deli / Bodega
1,Strathcona / Chinatown / Downtown Eastside,49.2779,-123.0908,Finch’s Market,49.278565,-123.093473,Sandwich Place
2,Strathcona / Chinatown / Downtown Eastside,49.2779,-123.0908,MacLean Park,49.278809,-123.088546,Park
3,Strathcona / Chinatown / Downtown Eastside,49.2779,-123.0908,The Pie Shoppe,49.278286,-123.097104,Pie Shop
4,Strathcona / Chinatown / Downtown Eastside,49.2779,-123.0908,The Juice Truck,49.281281,-123.092120,Food Truck


In [13]:
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")
vancouver_onehot['Neighborhood'] = vancouver_venues['Neighborhood'] 
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]
vancouver_onehot.head()

,Neighborhood,ATM,Accessories Store,Airport,Airport Terminal,American Restaurant,Amphitheater,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I only need venues that are restaurants ... 

In [14]:
vancouver_resta = vancouver_onehot.loc[:, vancouver_onehot.columns.str.contains('neighborhood|restaurant|bodega|deli', case=False)]

In [15]:
vancouver_resta.head(10)

,Neighborhood,American Restaurant,Asian Restaurant,Australian Restaurant,Belgian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,...,Restaurant,Seafood Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Strathcona / Chinatown / Downtown Eastside,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Strathcona / Chinatown / Downtown Eastside,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [16]:
vancouver_grouped = vancouver_resta.groupby('Neighborhood').mean().reset_index()
vancouver_grouped_clustering = vancouver_grouped.drop('Neighborhood', 1)
vancouver_grouped.head()

,Neighborhood,American Restaurant,Asian Restaurant,Australian Restaurant,Belgian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,...,Restaurant,Seafood Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Bentall Centre,0.058824,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.058824,0.000000,0.000000
1,Central Kitsilano / Greektown,0.000000,0.0,0.0,0.0,0.00000,0.0,0.032258,0.0,0.016129,...,0.016129,0.0,0.0,0.016129,0.016129,0.0,0.0,0.000000,0.016129,0.016129
2,East Fairview / South Cambie,0.000000,0.0,0.0,0.0,0.03125,0.0,0.031250,0.0,0.000000,...,0.031250,0.0,0.0,0.000000,0.062500,0.0,0.0,0.000000,0.000000,0.000000
3,East Mount Pleasant,0.021277,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,...,0.021277,0.0,0.0,0.000000,0.063830,0.0,0.0,0.000000,0.000000,0.063830
4,Killarney,0.000000,0.0,0.0,0.0,0.00000,0.0,0.080000,0.0,0.040000,...,0.000000,0.0,0.0,0.000000,0.040000,0.0,0.0,0.000000,0.000000,0.000000


Let's print each neighborhood along with the top 5 most common venues

In [17]:
num_top_venues = 5

for hood in vancouver_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = vancouver_grouped[vancouver_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bentall Centre----
                 venue  freq
0  American Restaurant  0.06
1   Turkish Restaurant  0.06
2     Ramen Restaurant  0.00
3  Lebanese Restaurant  0.00
4     Malay Restaurant  0.00


----Central Kitsilano / Greektown----
                             venue  freq
0                Indian Restaurant  0.03
1               Chinese Restaurant  0.03
2            Vietnamese Restaurant  0.02
3  Southern / Soul Food Restaurant  0.02
4               Mexican Restaurant  0.02


----East Fairview / South Cambie----
                  venue  freq
0      Sushi Restaurant  0.06
1      Malay Restaurant  0.03
2  Cantonese Restaurant  0.03
3    Chinese Restaurant  0.03
4            Restaurant  0.03


----East Mount Pleasant----
                   venue  freq
0  Vietnamese Restaurant  0.06
1       Sushi Restaurant  0.06
2   Ethiopian Restaurant  0.04
3             Restaurant  0.02
4    American Restaurant  0.02


----Killarney----
                  venue  freq
0    Chinese Restaurant  0.08
1 

In [18]:
kclusters = 6
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vancouver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 1, 3, 5, 2, 3, 3], dtype=int32)

In [19]:
vancouver_grouped_clustering.head()

,American Restaurant,Asian Restaurant,Australian Restaurant,Belgian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Dim Sum Restaurant,...,Restaurant,Seafood Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0.058824,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.058824,0.000000,0.000000
1,0.000000,0.0,0.0,0.0,0.00000,0.0,0.032258,0.0,0.016129,0.0,...,0.016129,0.0,0.0,0.016129,0.016129,0.0,0.0,0.000000,0.016129,0.016129
2,0.000000,0.0,0.0,0.0,0.03125,0.0,0.031250,0.0,0.000000,0.0,...,0.031250,0.0,0.0,0.000000,0.062500,0.0,0.0,0.000000,0.000000,0.000000
3,0.021277,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,...,0.021277,0.0,0.0,0.000000,0.063830,0.0,0.0,0.000000,0.000000,0.063830
4,0.000000,0.0,0.0,0.0,0.00000,0.0,0.080000,0.0,0.040000,0.0,...,0.000000,0.0,0.0,0.000000,0.040000,0.0,0.0,0.000000,0.000000,0.000000


Function to sort the venues , 10 most common, in descending order and create a dataframe. 

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = vancouver_grouped['Neighborhood']

for ind in np.arange(vancouver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vancouver_grouped.iloc[ind, :], num_top_venues)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

vancouver_merged = neighborhoods

#  add latitude/longitude for each neighborhood
vancouver_merged = vancouver_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
vancouver_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V6A,Vancouver,Strathcona / Chinatown / Downtown Eastside,49.2779,-123.091,3,Vietnamese Restaurant,Seafood Restaurant,Restaurant,Deli / Bodega,Asian Restaurant,Belgian Restaurant,Falafel Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant
1,V6B,Vancouver,NE Downtown / Gastown / Harbour Centre / Inter...,49.2788,-123.114,3,Restaurant,Vietnamese Restaurant,Sushi Restaurant,Fast Food Restaurant,Ramen Restaurant,Italian Restaurant,American Restaurant,Lebanese Restaurant,Seafood Restaurant,Asian Restaurant
2,V6C,Vancouver,Waterfront / Coal Harbour / Canada Place,49.2866,-123.116,3,Restaurant,Vietnamese Restaurant,Seafood Restaurant,Vegetarian / Vegan Restaurant,New American Restaurant,American Restaurant,Italian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Falafel Restaurant
3,V6E,Vancouver,SE West End / Davie Village,49.2833,-123.13,3,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Restaurant,Mexican Restaurant,Falafel Restaurant,Greek Restaurant,Indian Restaurant,Malay Restaurant
4,V6G,Vancouver,NW West End / Stanley Park,49.299,-123.141,3,Vietnamese Restaurant,Ethiopian Restaurant,Hotpot Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

import matplotlib.cm as cm
import matplotlib.colors as colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vancouver_merged['Latitude'], vancouver_merged['Longitude'], vancouver_merged['Neighborhood'], vancouver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

* Not specific restaurant cluster:

In [22]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 0, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,SE Oakridge / East Marpole / South Sunset,0,Restaurant,Indian Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant


* Chinese restaurant cluster:

In [23]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 1, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,North Grandview-Woodland,1,Italian Restaurant,Chinese Restaurant,Deli / Bodega,Indian Restaurant,Japanese Restaurant,Asian Restaurant,Vietnamese Restaurant,Sushi Restaurant,Thai Restaurant,South Indian Restaurant
23,SE Kerrisdale / SW Oakridge / West Marpole,1,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Sushi Restaurant,Restaurant,Dim Sum Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Thai Restaurant
24,SE Kerrisdale / SW Oakridge / West Marpole,1,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Sushi Restaurant,Restaurant,Dim Sum Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Thai Restaurant
28,Killarney,1,Chinese Restaurant,Deli / Bodega,Sushi Restaurant,Fast Food Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant
34,West Kensington-Cedar Cottage / NE Riley Park-...,1,Chinese Restaurant,Vietnamese Restaurant,Filipino Restaurant,Restaurant,Greek Restaurant,Indian Restaurant,Japanese Restaurant,Malay Restaurant,American Restaurant,Seafood Restaurant
35,West Kensington-Cedar Cottage / NE Riley Park-...,1,Chinese Restaurant,Vietnamese Restaurant,Filipino Restaurant,Restaurant,Greek Restaurant,Indian Restaurant,Japanese Restaurant,Malay Restaurant,American Restaurant,Seafood Restaurant
36,SE Riley Park-Little Mountain / SW Kensington-...,1,Chinese Restaurant,Indian Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Asian Restaurant,Tapas Restaurant,Japanese Restaurant,Cantonese Restaurant,Himalayan Restaurant,Hawaiian Restaurant
37,SE Riley Park-Little Mountain / SW Kensington-...,1,Chinese Restaurant,Indian Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Asian Restaurant,Tapas Restaurant,Japanese Restaurant,Cantonese Restaurant,Himalayan Restaurant,Hawaiian Restaurant


* not specific but with dim sum flavor:

In [24]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 2, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,NW Dunbar-Southlands / Chaldecutt / South Univ...,2,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
30,NW Dunbar-Southlands / Chaldecutt / South Univ...,2,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


* asian restaurant:

In [25]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 3, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Strathcona / Chinatown / Downtown Eastside,3,Vietnamese Restaurant,Seafood Restaurant,Restaurant,Deli / Bodega,Asian Restaurant,Belgian Restaurant,Falafel Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant
1,NE Downtown / Gastown / Harbour Centre / Inter...,3,Restaurant,Vietnamese Restaurant,Sushi Restaurant,Fast Food Restaurant,Ramen Restaurant,Italian Restaurant,American Restaurant,Lebanese Restaurant,Seafood Restaurant,Asian Restaurant
2,Waterfront / Coal Harbour / Canada Place,3,Restaurant,Vietnamese Restaurant,Seafood Restaurant,Vegetarian / Vegan Restaurant,New American Restaurant,American Restaurant,Italian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Falafel Restaurant
3,SE West End / Davie Village,3,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Restaurant,Mexican Restaurant,Falafel Restaurant,Greek Restaurant,Indian Restaurant,Malay Restaurant
4,NW West End / Stanley Park,3,Vietnamese Restaurant,Ethiopian Restaurant,Hotpot Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
5,NW West End / Stanley Park,3,Vietnamese Restaurant,Ethiopian Restaurant,Hotpot Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
6,West Fairview / Granville Island / NE Shaughnessy,3,Japanese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
7,West Fairview / Granville Island / NE Shaughnessy,3,Japanese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
8,NW Shaughnessy / East Kitsilano / Quilchena,3,Japanese Restaurant,Vietnamese Restaurant,Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Hotpot Restaurant,Thai Restaurant,Sushi Restaurant,Cantonese Restaurant
9,NW Shaughnessy / East Kitsilano / Quilchena,3,Japanese Restaurant,Vietnamese Restaurant,Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Hotpot Restaurant,Thai Restaurant,Sushi Restaurant,Cantonese Restaurant


* chinese food:

In [26]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 4, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,South Shaughnessy / NW Oakridge / NE Kerrisdal...,4,Chinese Restaurant,Asian Restaurant,Sushi Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant
18,South Shaughnessy / NW Oakridge / NE Kerrisdal...,4,Chinese Restaurant,Asian Restaurant,Sushi Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant
40,West Mount Pleasant / West Riley Park-Little M...,4,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant


* italian food:

In [27]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 5, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,NW Arbutus Ridge / NE Dunbar-Southlands,5,Italian Restaurant,Japanese Restaurant,Caribbean Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
15,NW Arbutus Ridge / NE Dunbar-Southlands,5,Italian Restaurant,Japanese Restaurant,Caribbean Restaurant,Ethiopian Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
